In [25]:
"""
Mask R-CNN
Configurations and data loading code for MS COCO.

Copyright (c) 2017 Matterport, Inc.
Licensed under the MIT License (see LICENSE for details)
Written by Waleed Abdulla

------------------------------------------------------------

Usage: import the module (see Jupyter notebooks for examples), or run from
       the command line as such:

    # Train a new model starting from pre-trained COCO weights
    python3 coco.py train --dataset=/path/to/coco/ --model=coco

    # Train a new model starting from ImageNet weights
    python3 coco.py train --dataset=/path/to/coco/ --model=imagenet

    # Continue training a model that you had trained earlier
    python3 coco.py train --dataset=/path/to/coco/ --model=/path/to/weights.h5

    # Continue training the last model you trained
    python3 coco.py train --dataset=/path/to/coco/ --model=last

    # Run COCO evaluatoin on the last model you trained
    python3 coco.py evaluate --dataset=/path/to/coco/ --model=last
"""

import os
import time
import numpy as np
%load_ext autoreload
%autoreload 1
print(' breakpoint 1')
# Download and install the Python COCO tools from https://github.com/waleedka/coco
# That's a fork from the original https://github.com/pdollar/coco with a bug
# fix for Python 3.
# I submitted a pull request https://github.com/cocodataset/cocoapi/pull/50
# If the PR is merged then use the original repo.
# Note: Edit PythonAPI/Makefile and replace "python" with "python3".
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

from   mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.dataset as dataset

# Root directory of the project
ROOT_DIR = os.getcwd()

# Path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")



 breakpoint 1


In [31]:
from coco import CocoDataset, CocoConfig, evaluate_coco, build_coco_results
import pprint as pp
pp = pp.PrettyPrinter(indent=4)
import argparse

# Parse command line arguments
parser = argparse.ArgumentParser(description='Train Mask R-CNN on MS COCO.')
parser.add_argument("command",
                    metavar="<command>",
                    help="'train' or 'evaluate' on MS COCO")
parser.add_argument('--dataset', required=True,
                    metavar="/path/to/coco/",
                    help='Directory of the MS-COCO dataset')
parser.add_argument('--model', required=True,
                    metavar="/path/to/weights.h5",
                    help="Path to weights .h5 file or 'coco'")
parser.add_argument('--logs', required=False,
                    default=DEFAULT_LOGS_DIR,
                    metavar="/path/to/logs/",
                    help='Logs and checkpoints directory (default=logs/)')
parser.add_argument('--limit', required=False,
                    default=500,
                    metavar="<image count>",
                    help='Images to use for evaluation (defaults=500)')
# args = parser.parse_args()
args = parser.parse_args("train --dataset E:\MLDatasets\coco2014 --model mask_rcnn_coco.h5 --limit 10".split())
# pp.pprint(options)
pp.pprint(args)
print("Model:   ", args.model)
print("Dataset: ", args.dataset)
print("Logs:    ", args.logs)
print("Limit:   ", args.limit)

# Configurations

Namespace(command='train', dataset='E:\\MLDatasets\\coco2014', limit='10', logs='E:\\git_projs\\Mask_RCNN_2\\logs', model='mask_rcnn_coco.h5')
Model:    mask_rcnn_coco.h5
Dataset:  E:\MLDatasets\coco2014
Logs:     E:\git_projs\Mask_RCNN_2\logs
Limit:    10


In [32]:
# Configurations
if args.command == "train":
    config = CocoConfig()
else:
    class InferenceConfig(CocoConfig):
        # Set batch size to 1 since we'll be running inference on
        # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
        DETECTION_MIN_CONFIDENCE = 0
    config = InferenceConfig()
config.display()

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
# tf_config = tf.ConfigProto()
# tf_config.gpu_options.per_process_gpu_memory_fraction = 0.55
# set_session(tf.Session(config=tf_config))

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.55)
set_session(tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)))


Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINING     

In [ ]:
# Create model
if args.command == "train":
    print('setup model for Training ')
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=args.logs)
else:
    print('setup model for Inference ')
    model = modellib.MaskRCNN(mode="inference", config=config,
                              model_dir=args.logs)

setup model for Training 
 IMAGE SHAPE is : 1024    1024


In [39]:
# for layer in model.keras_model.layers
#     print layer.name 
mdl = model.keras_model
layers    = [layer for layer in mdl.layers]          # all layer outputs
len(layers)

406

In [68]:
print(' layer {} :  {}'.format(idx,layers[0]))
pp.pprint(layers[0]._inbound_nodes[0].__dict__)
pp.pprint(layers[0].__dict__)
print('-------------------------------------------')

 layer 405 :  <keras.engine.topology.InputLayer object at 0x000002E98F24DEF0>
{   'arguments': None,
    'inbound_layers': [],
    'input_masks': [None],
    'input_shapes': [(None, 1024, 1024, 3)],
    'input_tensors': [   <tf.Tensor 'input_image_2:0' shape=(?, 1024, 1024, 3) dtype=float32>],
    'node_indices': [],
    'outbound_layer': <keras.engine.topology.InputLayer object at 0x000002E98F24DEF0>,
    'output_masks': [None],
    'output_shapes': [(None, 1024, 1024, 3)],
    'output_tensors': [   <tf.Tensor 'input_image_2:0' shape=(?, 1024, 1024, 3) dtype=float32>],
    'tensor_indices': []}
{   '_built': True,
    '_inbound_nodes': [   <keras.engine.topology.Node object at 0x000002E98F21BBE0>],
    '_initial_weights': None,
    '_losses': [],
    '_non_trainable_weights': [],
    '_outbound_nodes': [   <keras.engine.topology.Node object at 0x000002E988669828>],
    '_per_input_losses': {},
    '_per_input_updates': {},
    '_trainable_weights': [],
    '_updates': [],
    'batch_i

In [121]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
for idx,layer in enumerate(layers):
    print('>layer {} : name : {}  type: {}'.format(idx,layer.name,layer))
#     pp.pprint(layer._inbound_nodes[0].__dict__)
    print(">> input_shape {} inbound node {} ".format(layer._inbound_nodes[0].input_shapes,layer._inbound_nodes[0]))
#     print('>> inbound node output_shape {}'.format(layer._inbound_nodes[0].output_shapes))
#     if isinstance("layer", mrcnn.model.BatchNorm):
#         print('Batch Nomraliation layer')
    if 'padding' in layer.__dict__:
        print('\t padding : ',layer.padding)
    if 'filters' in layer.__dict__:
        print('\t Filters : ',layer.filters, ' Kernel Size :', layer.kernel_size, 'Stride:', layer.strides, ' Shape',layer.kernel.shape)
    
    if 'activation' in layer.__dict__:
        print('\t ',layer.activation)
    if 'pool_size' in layer.__dict__:
        print('\t poolsize : ',layer.pool_size,'Stride:', layer.strides)

#     pp.pprint(layer.__dict__)
    print("<< outbound shape{} input_shape {}".format(layer._outbound_nodes[0].input_shapes,layer._outbound_nodes[0]))
#     print('<< outbound node output_shape {}'.format(layer._outbound_nodes[0].output_shapes))
    print('-------------------------------------------')
    #layer.name, layer.input._keras_shape, layer.output._keras_shape

>layer 0 : name : input_image  type: <keras.engine.topology.InputLayer object at 0x000002E98F24DEF0>
>> input_shape [(None, 1024, 1024, 3)] inbound node <keras.engine.topology.Node object at 0x000002E98F21BBE0> 
<< outbound shape[(None, 1024, 1024, 3)] input_shape <keras.engine.topology.Node object at 0x000002E988669828>
-------------------------------------------
>layer 1 : name : zero_padding2d_3  type: <keras.layers.convolutional.ZeroPadding2D object at 0x000002E98F2250B8>
>> input_shape [(None, 1024, 1024, 3)] inbound node <keras.engine.topology.Node object at 0x000002E988669828> 
	 padding :  ((3, 3), (3, 3))
<< outbound shape[(None, 1030, 1030, 3)] input_shape <keras.engine.topology.Node object at 0x000002E988675780>
-------------------------------------------
>layer 2 : name : conv1  type: <keras.layers.convolutional.Conv2D object at 0x000002E98F225CC0>
>> input_shape [(None, 1030, 1030, 3)] inbound node <keras.engine.topology.Node object at 0x000002E988675780> 
	 padding :  val

IndexError: list index out of range